In [68]:
from project.models import Project, Emprise, Plan, PlanEmprise
from public_data.models import CommunesSybarval as Comm
from django.contrib.gis.db.models import Union
from django.contrib.gis.db.models.functions import Intersection, Area, Transform
from users.models import User
from django.conf import settings
from django.core.serializers import serialize
from prettytable import PrettyTable
from django.db.models import F

default_user = User.objects.get(email__startswith="swann")

In [60]:
# show available project to let you choose an id

x = PrettyTable(field_names=["id", "Nom du projet"], align="l")
x.add_rows([(p.id, p.name) for p in Project.objects.all().order_by("id")])
print(x)

+----+---------------+
| id | Nom du projet |
+----+---------------+
| 1  | Eyre          |
| 2  | COBAS         |
| 3  | Test geo      |
| 4  | COBAS_SHAPE   |
+----+---------------+


In [64]:
# get project and combined emprise
test_project = Project.objects.get(pk=2)
print(f"Project: {test_project.name}")
print("=========================")
geom = test_project.combined_emprise

test_filters = ["coveredby", "within", "contained", "crosses", "disjoint", "intersects", "overlaps"]
# contained : boss avec le rectangle englobant

communes = {_._commune: [] for _ in Comm.objects.all()}

for t in test_filters:
    for city_name in communes.keys():
        communes[city_name].append("")
    for city in Comm.objects.filter(**{f"mpoly__{t}": geom}):
        communes[city._commune][-1] = "OUI"

x = PrettyTable(field_names=["Commune"]+test_filters, align="l")
for name, row in communes.items():
    x.add_row([name]+row)
print(x)

Project: COBAS
+--------------------+-----------+--------+-----------+---------+----------+------------+----------+
| Commune            | coveredby | within | contained | crosses | disjoint | intersects | overlaps |
+--------------------+-----------+--------+-----------+---------+----------+------------+----------+
| Belin-Beliet       |           |        |           |         | OUI      |            |          |
| Salles             |           |        |           |         |          | OUI        |          |
| Audenge            |           |        |           |         | OUI      |            |          |
| Marcheprime        |           |        |           |         | OUI      |            |          |
| Andernos-les-Bains |           |        |           |         | OUI      |            |          |
| Gujan-Mestras      | OUI       | OUI    | OUI       |         |          | OUI        |          |
| Ares               |           |        |           |         | OUI      |

In [33]:
from rest_framework_gis.serializers import GeoFeatureModelSerializer, GeometryField
from rest_framework import serializers

class CustomSerializer(GeoFeatureModelSerializer):
    # intersection_area = serializers.FloatField()
    intersection = GeometryField()
    
    class Meta:
        model = Comm
        geo_field = "intersection"
        fields=('_commune', )

test_project = Project.objects.get(pk=4)
geom = test_project.combined_emprise
qs = Comm.objects.filter(mpoly__intersects=geom)
qs = qs.annotate(intersection=Intersection("mpoly", geom))
qs = qs.annotate(intersection_area=Area("intersection"))
qs = qs.annotate(area=Area("mpoly"))
for city in qs:
    print(city._commune, city.area /  1000000, city.intersection_area)
    
serializer = CustomSerializer(qs, many=True)
    
data = serializer.data
# data

Salles 1.5660665856949803e-08 sq_m 3.4531283614048655e-22 sq_m
Gujan-Mestras 5.733100597698255e-09 sq_m 0.005733080526963266 sq_m
Arcachon 8.739505356990526e-10 sq_m 0.0008739417713880599 sq_m
Le Teich 9.744691378456302e-09 sq_m 0.009744663421105871 sq_m
La Teste-de-Buch 2.0035415737886224e-08 sq_m 0.020035362010338183 sq_m
Mios 1.557048535973245e-08 sq_m 1.1700839029969788e-08 sq_m
Biganos 5.805485982194225e-09 sq_m 4.044118039452189e-09 sq_m


In [27]:
import os
import json
import random
import requests

from ipyleaflet import Map, GeoJSON


def random_color(feature):
    return {
        'color': 'black',
        'fillColor': random.choice(['red', 'yellow', 'green', 'orange']),
    }

m = Map(center=(44.6586, -1.164), zoom=11)

geo_json = GeoJSON(
    data=data,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },
    style_callback=random_color
)
m.add_layer(geo_json)

m

Map(center=[44.6586, -1.164], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [32]:
# lambert 93 - EPSG 2154
mios = Comm.objects.get(_commune="Mios")

def get_km_sq(mpoly): 
    """ 
    Returns the area in acres. 
    """ 
    # Convert our geographic polygons (in WGS84)
    # into a local projection for France (here lambert 93 - EPSG 2154) 
    mpoly.transform(2154)
    # we are in m² and we want km²
    return mpoly.area / (1000 * 1000)

print(get_km_sq(mios.mpoly))

137.21781664992756


In [71]:
test_project = Project.objects.get(pk=4)
geom = test_project.combined_emprise
qs = Comm.objects.filter(mpoly__intersects=geom)
qs = qs.annotate(mpoly_2=Transform("mpoly", 2154))
# qs = qs.annotate(intersection=Intersection("mpoly", geom))
# qs = qs.annotate(intersection2=Transform("intersection", 2154))
qs = qs.annotate(intersection=Transform(Intersection("mpoly", geom), 2154))
qs = qs.annotate(intersection_area=Area("intersection"))
qs = qs.annotate(area=Area("mpoly_2"))

x = PrettyTable()
x.field_names = ["Commune", "Aire (km²)", "Aire (km²) intersection COBAS", ]
x.align = "r"
x.align["Commune"] = "l"
for city in qs:
    x.add_row([city._commune, f"{city.area.sq_km:.2f}", f"{city.intersection_area.sq_km:.2f}"])
print(x)

+------------------+------------+-------------------------------+
| Commune          | Aire (km²) | Aire (km²) intersection COBAS |
+------------------+------------+-------------------------------+
| Salles           |     138.20 |                          0.00 |
| Gujan-Mestras    |      50.55 |                         50.55 |
| Arcachon         |       7.70 |                          7.70 |
| Le Teich         |      85.94 |                         85.94 |
| La Teste-de-Buch |     176.76 |                        176.76 |
| Mios             |     137.22 |                          0.00 |
| Biganos          |      51.12 |                          0.00 |
+------------------+------------+-------------------------------+


In [69]:
qs = qs.filter(intersection_area__gt=F("area") * 0.95 )

x = PrettyTable()
x.field_names = ["Commune", "Aire (km²)", "Aire (km²) intersection COBAS", ]
x.align = "r"
x.align["Commune"] = "l"
for city in qs:
    x.add_row([city._commune, f"{city.area.sq_km:.2f}", f"{city.intersection_area.sq_km:.2f}"])
print(x)

+------------------+------------+-------------------------------+
| Commune          | Aire (km²) | Aire (km²) intersection COBAS |
+------------------+------------+-------------------------------+
| Arcachon         |       7.70 |                          7.70 |
| Le Teich         |      85.94 |                         85.94 |
| Gujan-Mestras    |      50.55 |                         50.55 |
| La Teste-de-Buch |     176.76 |                        176.76 |
+------------------+------------+-------------------------------+


In [66]:
from django.db import connection
qs = Comm.objects.filter(code_insee__in=["33527", ])
qs = qs.aggregate(mpoly=Union("mpoly"))
connection.queries.pop()

{'sql': 'SELECT ST_Union("public_data_communessybarval"."mpoly")::bytea AS "mpoly" FROM "public_data_communessybarval" WHERE "public_data_communessybarval"."code_insee" IN (\'33527\')',
 'time': '0.001'}

TypeError: Object of type Area is not JSON serializable

In [3]:
evaluate_couverture_and_usage.delay(p)

<Project: COBAS>